In [1]:
import gc
import torch
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding, DataCollatorForLanguageModeling
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
import transformers
import os

gc.collect()
torch.cuda.empty_cache()

In [2]:

# List of glue tasks
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

#List of glue keys
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

#Select task
task = "rte"  #cola, mrpc
batch_size = 10 #10 normally, 8 for qnli

# Load dataset based on task variable
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

#Collect sentence keys and labels
sentence1_key, sentence2_key = task_to_keys[task]

# Number of logits to output
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

Reusing dataset glue (/home/ubuntu/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


## Load Deberta

In [3]:
###  Tokenizing Section  ####

#Load model
model_checkpoint = "microsoft/deberta-v3-small"

# Create tokenizer for respective model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)

def tokenizer_func(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True,)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)

# tokenize sentence(s)
encoded_dataset = dataset.map(tokenizer_func, batched=True)

#model_checkpoint = "deberta-v3-small_baseline_cola/"
model_checkpoint = "deberta-v3-small_baseline_"+actual_task+"/"

###  Model Section  ####

# Create model and attach ForSequenceClassification head
model_deberta = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# Type of metric for given task
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    f"{model_checkpoint}-finetuned-Testing-{task}",
    evaluation_strategy = "epoch",
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=5
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model_deberta,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

The following columns in the evaluation set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 10


{'eval_loss': 0.6397655010223389,
 'eval_accuracy': 0.6678700361010831,
 'eval_runtime': 2.6819,
 'eval_samples_per_second': 103.286,
 'eval_steps_per_second': 10.441}

In [4]:
### Collect Predictions  ###

prediction_deberta = trainer.predict(encoded_dataset[validation_key])

The following columns in the test set  don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 277
  Batch size = 10


In [5]:
prediction_deberta

PredictionOutput(predictions=array([[-2.28477661e-02,  1.31481946e-01],
       [-1.21065445e-01,  1.81684196e-01],
       [ 1.68376476e-01, -8.62409621e-02],
       [-3.04296557e-02,  1.09341688e-01],
       [ 5.24833500e-01, -4.99871224e-01],
       [ 2.20424160e-01, -1.45396665e-01],
       [ 2.09858373e-01, -1.01460971e-01],
       [ 3.95472109e-01, -3.66178989e-01],
       [ 1.20707341e-01, -2.10909601e-02],
       [ 4.75655973e-01, -4.26238716e-01],
       [ 5.38933158e-01, -5.25938869e-01],
       [ 1.87906191e-01, -1.17753774e-01],
       [ 1.03429981e-01,  1.74361039e-02],
       [ 4.99124467e-01, -4.46061015e-01],
       [ 5.13817787e-01, -4.47030008e-01],
       [-1.73950121e-01,  2.43124902e-01],
       [ 7.50662506e-01, -6.77986801e-01],
       [ 4.24016178e-01, -3.96278471e-01],
       [ 2.12285016e-02,  7.43411630e-02],
       [-1.65889665e-01,  2.26274475e-01],
       [ 4.98478949e-01, -4.54192132e-01],
       [ 9.14392099e-02,  1.26467913e-03],
       [ 1.87638775e-01, 

In [6]:
## Clear the Cache
gc.collect()
torch.cuda.empty_cache()

## Load Electra

In [7]:
###  Tokenizing Section  ####

#Load model
model_checkpoint = "google/electra-small-discriminator"

# Create tokenizer for respective model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)

def tokenizer_func(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True,)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)

# tokenize sentence(s)
encoded_dataset = dataset.map(tokenizer_func, batched=True)

#model_checkpoint = "electra-small-discriminator-finetuned-cola/"
#model_checkpoint = "Electra_fintuned_cola/"
model_checkpoint = "Electra_fintuned_"+actual_task+"/"

###  Model Section  ####

# Create model and attach ForSequenceClassification head
model_electra = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# Type of metric for given task
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"


args = TrainingArguments(
    f"{model_checkpoint}-finetuned-Testing-{task}",
    evaluation_strategy = "epoch",
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=5
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model_electra,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()


loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading file 

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

loading configuration file Electra_fintuned_rte/config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "type_vocab_size": 2,
  "vocab_size": 30522
}

loading weights file Electra_fintuned_rte/pytorch_model.bin
All model checkpoint weights were used when ini

{'eval_loss': 0.9825196862220764,
 'eval_accuracy': 0.6823104693140795,
 'eval_runtime': 0.8045,
 'eval_samples_per_second': 344.306,
 'eval_steps_per_second': 34.804}

In [8]:
### Collect Predictions  ###
## Clear the Cache
gc.collect()
torch.cuda.empty_cache()
prediction_electra = trainer.predict(encoded_dataset[validation_key])

The following columns in the test set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 277
  Batch size = 10


In [9]:
## Clear the Cache
gc.collect()
torch.cuda.empty_cache()

In [10]:
prediction_electra

PredictionOutput(predictions=array([[ 2.02784848e+00, -1.85461128e+00],
       [-2.11592221e+00,  2.06107330e+00],
       [ 2.00161529e+00, -1.87093067e+00],
       [-1.13361764e+00,  1.06826031e+00],
       [ 2.01405787e+00, -1.90909684e+00],
       [ 2.17983341e+00, -2.02379394e+00],
       [ 2.06975746e+00, -1.97160697e+00],
       [ 2.12865710e+00, -1.99404538e+00],
       [ 2.14317846e+00, -1.97988260e+00],
       [ 2.11348605e+00, -2.02246380e+00],
       [ 2.12435722e+00, -2.00981998e+00],
       [ 1.87493932e+00, -1.79534864e+00],
       [-1.53496981e-01,  1.05426125e-01],
       [ 2.07774019e+00, -1.89864492e+00],
       [ 1.55826092e+00, -1.62164414e+00],
       [-1.33845544e+00,  1.31783879e+00],
       [ 2.16355371e+00, -2.01471734e+00],
       [ 2.17020321e+00, -2.02563715e+00],
       [ 2.10215902e+00, -1.99980223e+00],
       [-2.23491168e+00,  2.20087075e+00],
       [ 2.01277375e+00, -1.89228380e+00],
       [ 5.86494088e-01, -6.35950685e-01],
       [ 1.91349852e+00, 

In [11]:
print('done')

done


## Load XLNet

In [12]:
###  Tokenizing Section  ####

#Load model
model_checkpoint = "xlnet-base-cased"

# Create tokenizer for respective model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, truncation=True, model_max_length=512)

def tokenizer_func(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True,)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True,)

# tokenize sentence(s)
encoded_dataset = dataset.map(tokenizer_func, batched=True)

#model_checkpoint = "electra-small-discriminator-finetuned-cola/"
#model_checkpoint = "Electra_fintuned_cola/"
model_checkpoint = "xlnet-base-cased_baseline_"+actual_task+"/"

###  Model Section  ####

# Create model and attach ForSequenceClassification head
model_xlnet = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

# Type of metric for given task
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"


args = TrainingArguments(
    f"{model_checkpoint}-finetuned-Testing-{task}",
    evaluation_strategy = "epoch",
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    metric_for_best_model=metric_name,
    eval_accumulation_steps=5
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model_xlnet,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlnet-base-cased/resolve/main/config.json from cache at /home/ubuntu/.cache/huggingface/transformers/06bdb0f5882dbb833618c81c3b4c996a0c79422fa2c95ffea3827f92fc2dba6b.da982e2e596ec73828dbae86525a1870e513bd63aae5a2dc773ccc840ac5c346
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": tr

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

loading configuration file xlnet-base-cased_baseline_rte/config.json
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetForSequenceClassification"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "problem_type": "single_label_classification",
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.9.2",
  "untie_r": true,
  "u

{'eval_loss': 0.5314945578575134,
 'eval_accuracy': 0.740072202166065,
 'eval_runtime': 6.7372,
 'eval_samples_per_second': 41.115,
 'eval_steps_per_second': 4.156}

In [13]:
### Collect Predictions  ###
## Clear the Cache
gc.collect()
torch.cuda.empty_cache()
prediction_xlnet = trainer.predict(encoded_dataset[validation_key])

The following columns in the test set  don't have a corresponding argument in `XLNetForSequenceClassification.forward` and have been ignored: sentence1, sentence2, idx.
***** Running Prediction *****
  Num examples = 277
  Batch size = 10


In [14]:
## Clear the Cache
gc.collect()
torch.cuda.empty_cache()

In [15]:
prediction_xlnet

PredictionOutput(predictions=array([[-4.69099611e-01,  9.15669084e-01],
       [-1.20145805e-01, -2.79086847e-02],
       [ 1.07224667e+00, -7.68861651e-01],
       [-1.30934703e+00,  1.09649646e+00],
       [ 5.97861946e-01, -2.90515661e-01],
       [ 1.21624863e+00, -8.72906387e-01],
       [ 9.50232387e-01, -5.10504842e-01],
       [ 9.21778858e-01, -6.20690525e-01],
       [ 7.44625807e-01, -4.31579322e-01],
       [ 8.17010999e-01, -7.03544855e-01],
       [ 1.27667367e+00, -7.47535646e-01],
       [ 1.66172266e-01,  4.96861964e-01],
       [ 1.09488897e-01,  1.69696182e-01],
       [ 1.07658887e+00, -7.47908771e-01],
       [ 3.20169590e-02,  1.24714725e-01],
       [ 4.66568083e-01, -9.53244790e-02],
       [ 8.79339933e-01, -8.71043265e-01],
       [ 1.28256679e+00, -7.83891380e-01],
       [ 4.32303756e-01, -3.26966375e-01],
       [-1.25154042e+00,  1.40925264e+00],
       [ 2.39685863e-01, -1.17922537e-01],
       [-1.95877999e-01,  3.76305193e-01],
       [ 6.77487329e-02, 

# Random Forest

## Combine Model Predicions to create Input Features

In [16]:
import pandas as pd

#Labels
val_labels = prediction_deberta.label_ids


#DeBERTa
df_deberta = pd.DataFrame(prediction_deberta[0])
df_deberta=df_deberta.rename(columns=dict(zip(df_deberta.columns,['deberta_'+str(col) for col in df_deberta.columns])))
print(df_deberta.head(),'\n')


#Electra
df_electra = pd.DataFrame(prediction_electra[0])
df_electra=df_electra.rename(columns=dict(zip(df_electra.columns,['electra_'+str(col) for col in df_electra.columns])))
print(df_electra.head(),'\n')


#XLNet
df_xlnet = pd.DataFrame(prediction_xlnet[0])
df_xlnet=df_xlnet.rename(columns=dict(zip(df_xlnet.columns,['xlnet_'+str(col) for col in df_xlnet.columns])))
print(df_xlnet.head(),'\n')

   deberta_0  deberta_1
0  -0.022848   0.131482
1  -0.121065   0.181684
2   0.168376  -0.086241
3  -0.030430   0.109342
4   0.524834  -0.499871 

   electra_0  electra_1
0   2.027848  -1.854611
1  -2.115922   2.061073
2   2.001615  -1.870931
3  -1.133618   1.068260
4   2.014058  -1.909097 

    xlnet_0   xlnet_1
0 -0.469100  0.915669
1 -0.120146 -0.027909
2  1.072247 -0.768862
3 -1.309347  1.096496
4  0.597862 -0.290516 



In [17]:
#Combine the dataframes
df_combine = pd.concat([df_deberta, df_electra, df_xlnet], axis=1)
df_combine.head()

,deberta_0,deberta_1,electra_0,electra_1,xlnet_0,xlnet_1
0,-0.022848,0.131482,2.027848,-1.854611,-0.469100,0.915669
1,-0.121065,0.181684,-2.115922,2.061073,-0.120146,-0.027909
2,0.168376,-0.086241,2.001615,-1.870931,1.072247,-0.768862
3,-0.030430,0.109342,-1.133618,1.068260,-1.309347,1.096496
4,0.524834,-0.499871,2.014058,-1.909097,0.597862,-0.290516


In [18]:
# Importing the required packages

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [19]:
# Split the dataset into train and test

X_train, X_test, y_train, y_test = train_test_split(df_combine, val_labels, test_size=0.3, random_state=100)
X_train.head()

,deberta_0,deberta_1,electra_0,electra_1,xlnet_0,xlnet_1
259,0.222337,-0.132157,2.118173,-1.961455,0.700648,-0.347159
206,0.318081,-0.223418,-1.130229,1.141119,0.427897,-0.279077
48,0.479141,-0.472334,2.048319,-1.984463,0.941934,-0.595891
254,0.719674,-0.728521,1.700607,-1.635869,1.211807,-0.746192
200,-0.119261,0.191832,2.068220,-1.978909,0.921048,-0.665130


In [20]:
# Perform training with random forest with all columns
# Initialize random forest classifier
clf = RandomForestClassifier(n_estimators=100)

# Perform training
clf.fit(X_train, y_train)

# Predicton on test using all features
y_pred = clf.predict(X_test)
y_pred_score = clf.predict_proba(X_test)

In [21]:
metric_name

'accuracy'

In [22]:
# Print basic Report, then specify for the model

print("\n")
print("Results Using All Features: \n")

print("Classification Report: ")
print(classification_report(y_test,y_pred))
print("\n")

if metric_name == 'accuracy':
    print("Accuracy : ", accuracy_score(y_test, y_pred) * 100)
    
elif metric_name == 'matthews_correlation':
    print("Accuracy : ", matthews_corrcoef(y_test, y_pred) * 100)

elif metric_name == "pearson":
    print("Accuracy : ", matthews_corrcoef(y_test, y_pred) * 100)

else:
    print('ERROR')

print('-------------------')
print("DeBERTa : ", prediction_deberta.metrics['test_'+metric_name]*100)
print("Electra : ", prediction_electra.metrics['test_'+metric_name]*100)
print("XLNet : ", prediction_xlnet.metrics['test_'+metric_name]*100)



Results Using All Features: 

Classification Report: 
              precision    recall  f1-score   support

           0       0.73      0.68      0.71        44
           1       0.67      0.72      0.70        40

    accuracy                           0.70        84
   macro avg       0.70      0.70      0.70        84
weighted avg       0.70      0.70      0.70        84



Accuracy :  70.23809523809523
-------------------
DeBERTa :  66.78700361010831
Electra :  68.23104693140795
XLNet :  74.0072202166065


In [23]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat Dec  4 18:21:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla M60           Off  | 00000000:00:1E.0 Off |                    0 |
| N/

In [24]:
### How to make the test dataset not crash

#test2 = encoded_dataset['test'].remove_columns(['label'])
#new_column = [0] * len(test2)
#test3 = test2.add_column("label", new_column)

#Should run fine
#prediction_electra = trainer.predict(test3)
#prediction_electra[0]